# RNN on RNA K-mers

Extract every K-mer for K=3.
Apply one-hot encoding.
Convert to ragged tensors.
Train with 3-deep RNN.

Previous runs exhausted RAM.
This version trains the same model additively on small batches of data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import ShuffleSplit

import tensorflow as tf
from tensorflow import keras
tf.keras.backend.set_floatx('float32')  # save RAM


Load and encode the sequence data.

In [2]:
MIN_SEQ_LEN=200   ### by definition, lncRNA have min len 200
#MIN_SEQ_LEN=1000  ### we use this to reduce training RAM and CPU
MAX_SEQ_LEN=25000 ### this screens 4 outliers in the complete dataset
#MAX_SEQ_LEN=2000  ### we use this to reduce training RAM and CPU
DEFLINE='>'
ncfile='ncRNA.fasta' 
pcfile='pcRNA.fasta' 
#ncfile='tiny.ncRNA.fasta' # 10 seqs for faster debugging
#pcfile='tiny.pcRNA.fasta' # 10 seqs for faster debugging

#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
## !ls drive/'My Drive'/'Colab Notebooks'/*
#ncfile='drive/My Drive/Colab Notebooks/ncRNA.fasta'
#pcfile='drive/My Drive/Colab Notebooks/pcRNA.fasta'

K=3   # K-mer length = word length

# Assume file was preprocessed to contain one line per seq.
# Returned structure is ndarray of ndarray i.e no python lists inside.
def load_fasta(filename):
    seqs=[]
    with open (filename,'r') as infile:
        for line in infile:
            if line[0]!=DEFLINE and len(line)>=MIN_SEQ_LEN and len(line)<=MAX_SEQ_LEN:
                line=line.rstrip()
                linelen=len(line)
                kmers=[]
                for i in range(linelen-K+1): # e.g. k=3,|line=4| => range=2 so i={0,1}
                    kmer=line[i:i+K]
                    kmers.append(kmer)
                kmers=np.array(kmers)
                seqs.append(kmers.reshape(-1, 1)) # reshaped changes (any,) to (any,1)
    nparray=np.array(seqs)
    return nparray

print("Load "+ncfile)
nc_seqs = load_fasta(ncfile)
print("Load "+pcfile)
pc_seqs = load_fasta(pcfile)

encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
#seq=tf.reshape(nc_seqs[0],shape=(-1, 1)) # tensor flow version
seq=nc_seqs[0].reshape(-1, 1)
encoder.fit(seq)
print("Encoder categories")
print(str(encoder.categories_))

print("Ecode the non-coding sequences")
nc_list=[]
for seq in nc_seqs:
    encoded=encoder.transform(seq)  
    nc_list.append(encoded)
    
nc_all=np.array(nc_list) # .reshape(-1,1)
print("shape: "+str(nc_all.shape))
print("sequence 0 starts ")
print(nc_seqs[0][:7])
print("element 0 encoding: ")
print(nc_all[0][:5])

print("Encode the protein-coding sequences")
pc_list=[]
for seq in pc_seqs:
    encoded=encoder.transform(seq)
    pc_list.append(encoded)

pc_all=np.array(pc_list) # .reshape(-1,1)
pc_all.shape

Load ncRNA.fasta
Load pcRNA.fasta
Encoder categories
[array(['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA',
       'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC',
       'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG',
       'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT',
       'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA',
       'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC',
       'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG',
       'TTT'], dtype='<U3')]
Ecode the non-coding sequences
shape: (17711,)
sequence 0 starts 
[['TCA']
 ['CAT']
 ['ATC']
 ['TCA']
 ['CAG']
 ['AGT']
 ['GTC']]
element 0 encoding: 
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0

(20152,)

Create train and validation sets.

In [3]:
nc_labels=np.zeros(shape=(17711))
pc_labels=np.ones(shape=(20152))
#nc_labels=np.zeros(shape=(10))  # fast debugging
#pc_labels=np.ones(shape=(10))

#nc_labeled=np.concatenate((nc_all,nc_labels),axis=1)
#pc_labeled=np.concatenate((pc_all,pc_labels),axis=1)
all_seqs=np.concatenate((nc_all,pc_all),axis=0)
all_labels=np.concatenate((nc_labels,pc_labels),axis=0)

print("shape of sequences, shape of labels")
all_seqs.shape,all_labels.shape

splitter = ShuffleSplit(n_splits=1, test_size=0.2, random_state=37863)
for train_index,test_index in splitter.split(all_seqs):
    train_seqs =   all_seqs[train_index]
    train_labels = all_labels[train_index]
    test_seqs =    all_seqs[test_index]
    test_labels =  all_labels[test_index]
 
print("shape of train")
print(train_seqs.shape,train_labels.shape)
print("shape of test")
print(test_seqs.shape,test_labels.shape)

print("Convert numpy array to python 3D array")
def numpy_to_python_3D(np_seqs):
    one_set = []
    tlen = len(np_seqs)
    for i in range(tlen): # for every sequence in set
        one_seq = []
        slen = len(np_seqs[i])
        for j in range(slen): # for ever letter in sequence
            one_letter=np_seqs[i][j]
            one_seq.append(one_letter)
        one_set.append(one_seq)
    return one_set
train_seqs = numpy_to_python_3D(train_seqs)
test_seqs = numpy_to_python_3D(test_seqs)
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

# Now that values are shuffled, partition gives random sample.
data_size=len(train_seqs)
PARTITION=int(data_size*0.8)
print("Partition = "+str(PARTITION))

print("Partition train/validation")
X_train=train_seqs[:PARTITION]
X_valid=train_seqs[PARTITION:]
y_train=train_labels[:PARTITION]
y_valid=train_labels[PARTITION:]
print("Lengths of train,valid")
print((len(X_train),len(X_valid)))

print("Free memory.")
nc_seqs=None
pc_seqs=None
all_seqs=None
nc_labels=None
pc_labels=None
train_seqs=None
train_labels=None

shape of sequences, shape of labels
shape of train
(30290,) (30290,)
shape of test
(7573,) (7573,)
Convert numpy array to python 3D array
Partition = 24232
Partition train/validation
Lengths of train,valid
(24232, 6058)
Free memory.


In [5]:
print("Build the model")
seq_len=None  # none indicates variable length
input_features=4**K   # 64 DNA K-mers at K=3
rnn2 = keras.models.Sequential([
    keras.layers.SimpleRNN(64, return_sequences=True, input_shape=[seq_len,input_features]),
    keras.layers.SimpleRNN(64, return_sequences=True),
    keras.layers.SimpleRNN(64, return_sequences=True),
    keras.layers.SimpleRNN(1),
])

print("Build the training environment")
bc=tf.keras.losses.BinaryCrossentropy(from_logits=False)
rnn2.compile(loss=bc, optimizer="Adam",metrics=["accuracy"])
rnn2.summary()

Build the model
Build the training environment
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, None, 64)          8256      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 1)                 66        
Total params: 24,834
Trainable params: 24,834
Non-trainable params: 0
_________________________________________________________________


## Loop: additively train the model in small batches.

In [ ]:
EPOCHS=5
BATCHES=6
# batch max: 1K, 2K, 4K, 8K, 16K, 32K
for epoch in range(EPOCHS):
    print("Epoch %d"%epoch)
    batch_max = 0
    for batch in range(BATCHES):
        batch_min = batch_max
        batch_max = 1000 * (2**batch)
        print("Batch of RNA lengths %d to %d"%(batch_min,batch_max))
        X_batch_train = []
        y_batch_train = []
        for i in range(len(X_train)):
            L = len(X_train[i])
            if L>batch_min and L<=batch_max:
                X_batch_train.append(X_train[i])
                y_batch_train.append(y_train[i])
        X_batch_valid = []
        y_batch_valid = []
        for i in range(len(X_valid)):
            L = len(X_valid[i])
            if L>batch_min and L<=batch_max:
                X_batch_valid.append(X_valid[i])
                y_batch_valid.append(y_valid[i])
            
        print("Convert to tensors")
        tensor = tf.ragged.constant(X_batch_train)
        X_batch_train = tensor
        tensor=None
        print(type(X_batch_train))
        print(X_batch_train.shape)
        tensor = tf.convert_to_tensor(y_batch_train)
        y_batch_train = tensor
        tensor=None
        print(type(y_batch_train))
        print(y_batch_train.shape)
        tensor = tf.ragged.constant(X_batch_valid)
        X_batch_valid=tensor
        tensor=None
        print(type(X_batch_valid))
        print(X_batch_valid.shape)
        tensor = tf.convert_to_tensor(y_batch_valid)
        y_batch_valid = tensor
        tensor=None
        print(type(y_batch_valid))
        print(y_batch_valid.shape)
        
        print("Train the model")
        history = rnn2.fit(X_batch_train,y_batch_train,epochs=1,
                           validation_data=(X_batch_valid,y_batch_valid))

        print("Visualize training history.")
        pd.DataFrame(history.history).plot(figsize=(8,5))
        plt.grid(True)
        plt.gca().set_ylim(0,1)
        plt.show()

Epoch 0
Batch of RNA lengths 0 to 1000
Convert to tensors


Resources.
[Working with RNNs](https://keras.io/guides/working_with_rnns/).
[Recurrent Neural Networks with Keras](https://www.tensorflow.org/guide/keras/rnn#rnns_with_listdict_inputs_or_nested_inputs).
Function tf.convert_to_tensor [docs](https://www.tensorflow.org/api_docs/python/tf/convert_to_tensor).
Function tf.reshape [docs](https://www.tensorflow.org/api_docs/python/tf/reshape).
Ragged Tensors [tutorial](https://www.tensorflow.org/guide/tensor#ragged_tensors)
and [docs](https://www.tensorflow.org/api_docs/python/tf/RaggedTensor#documenting_raggedtensor_shapes_2) and [module](https://www.tensorflow.org/api_docs/python/tf/ragged).
Incredible speedup for convert to tensor by sirfz on [stackoverflow](https://stackoverflow.com/questions/44353509/tensorflow-tf-constant-initializer-is-very-slow).


